# Formateo de Datos (Tidy Data)
Como mencionamos anteriormente. Nos aprovecharemos de la forma de la base de datos para darle un formato más adecuado para su análisis. La estructura constará de usar un múlti indice en la parte de las columnas y de renglonres los municipios. 

## Extracción de Datos crudos (Raw Data)

In [125]:
import pandas as pd
import numpy as np

Sonora = pd.read_excel("..\\medium_data\\book_son.xlsx", sheet_name=None)
Municipios = list(Sonora.keys())

In [126]:
def transform(mun: str):
    df = Sonora[mun].copy()
    df.drop(columns=["Unnamed: 0.1","Unnamed: 0"], inplace=True)
    df.dropna(how = 'all', axis = 0, inplace=True)
    df.reset_index(drop=True,inplace=True)

    abs_index = []
    for i in range(len(df)):
        if df.iloc[i, 1] == 'Absoluto':
            abs_index.append(i)
    abs_index.append(len(df))
    new_dfs = []
    for a in range(len(abs_index) - 1):
        u = df.iloc[abs_index[a]:abs_index[a + 1],:]
        u = u.dropna(how = 'all', axis = 1) 
        new_dfs.append(u)
    return new_dfs


data_to_tidy ={}
for mun in Municipios:
    nl = []
    y = transform(mun)
    for df in y:
        df.columns = df.iloc[0,:]
        df = df.iloc[1:,:]
        df.reset_index(drop=True, inplace=True)
        nl.append(df)
    data_to_tidy[mun] = nl


 # Preliminares

In [127]:
import regex as re

def footnote(data: pd.DataFrame):
    extra_information = []
    extra_codes = ['Nota','^\d']
    remove_codes = ['Fuente']
    for i in list(data.iloc[:,0]):
        for code in extra_codes:
            if re.findall(code, i):
                extra_information.append(i)
                data = data.drop(index = i)
        for code in remove_codes:
            if re.findall(code, i):
                data = data.drop(index = list(data.iloc[:,0]).index(i))
    return data, extra_information


descriptor = pd.read_excel("..\\raw_data\\descriptor.xlsx")

descriptor['Origen'] = descriptor['Origen'].ffill()
descriptor.set_index(['Origen','Némonico'], drop=True, inplace=True)
sim_cats = descriptor.index.get_level_values(0).unique()

def filter_mi(cat):
    return descriptor.loc[(sim_cats[cat],slice(None)),:]

mun_0 = Municipios[0]

# Composición Territorial

In [128]:
test_to_tidy = data_to_tidy[mun_0]

ct  = test_to_tidy[0]
ct, _ = footnote(ct)

ct

2,Composición territorial,Absoluto,Porcentaje
0,Localidades,25,-
1,Urbanas,1,4
2,Rurales,24,96
3,AGEB,14,-
4,Urbanas,11,78.6
5,Rurales,3,21.4
6,Manzanas,223,-
7,Urbanas,135,60.5
8,Rurales,88,39.5


In [129]:
def abspercols(data):
    y = []
    for d in data:
        y += [d + 'A', d + '%']
    return y

cols = filter_mi(0).index.get_level_values(1)
cols 

comter = pd.DataFrame(index = Municipios, columns = abspercols(cols))
ap_01 = []
for i in range(len(ct)):
    ap_01 += list(ct.iloc[i,1:])
 
comter.iloc[0,:] = ap_01
comter

,LOCTOTA,LOCTOT%,LOCURBA,LOCURB%,LOCRURA,LOCRUR%,AGBTOTA,AGBTOT%,AGBURBA,AGBURB%,AGBRURA,AGBRUR%,MANTOTA,MANTOT%,MANURBA,MANURB%,MANRURA,MANRUR%
Aconchi,25,-,1,4,24,96,14,-,11,78.6,3,21.4,223,-,135,60.5,88,39.5
Agua Prieta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alamos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Altar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arivechi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Villa Pesqueira,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Yécora,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gral. Plutarco Elías Calles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Benito Juárez,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [130]:
def fill_mun(cat, mun):
    test_to_tidy = data_to_tidy[mun]
    ct  = test_to_tidy[cat]
    ct, _ = footnote(ct)
    ap_01 = []
    for i in range(len(ct)):
        ap_01 += list(ct.iloc[i,1:])
    return ap_01

def fill_df(cat):
    cols = filter_mi(cat).index.get_level_values(1)
    comter = pd.DataFrame(index = Municipios, columns = abspercols(cols))
    for m in Municipios:
        comter.loc[m,:] = fill_mun(cat, m)
    return comter

def mun_df(cols):
    return pd.DataFrame(index = Municipios, columns=cols)

comter = fill_df(0)

# 

In [136]:
import ubigeo as ugeo

ug = test_to_tidy[1]
ug, _= footnote(ug)
# geoson = pd.DataFrame(index = Municipios, columns= )
geo_cols = list(filter_mi(1).index.get_level_values(1))
geo_cols = geo_cols[:7] + abspercols(['SUPTRL']) + ['DENPOB']

def fill_geo(mun, cat, i: int, uf: pd.DataFrame):
    ug = data_to_tidy[mun]
    df  = ug[cat]
    df, _ = footnote(df)
    uf.iloc[i, 6] = df.loc[2,'Absoluto']
    uf.iloc[i, 7] = df.loc[3,'Absoluto']
    uf.iloc[i, 8] = df.loc[3,'Porcentaje']
    uf.iloc[i, 9] = df.loc[4,'Absoluto']
    coords = df.loc[0,'Absoluto']
    altura = df.loc[1,'Absoluto']
    latlonalt = ugeo.get_latlong(coords) + ugeo.get_altitud(altura)
    uf.iloc[i, :6] = latlonalt
    # print(mun)
  
ubigeo = mun_df(geo_cols)
for m,mun in enumerate(Municipios):
    fill_geo(mun, 1, m, ubigeo)
ubigeo

,LATMIN,LATMAX,LONMIN,LONMAX,ALTMIN,ALTMAX,COLINS,SUPTRLA,SUPTRL%,DENPOB
Aconchi,29.683333,29.883333,110.0,110.433333,500.0,2200.0,Colinda al norte con los municipios de San Fel...,368.2,0.2,7
Agua Prieta,30.7,31.35,108.683333,109.75,800.0,2500.0,Colinda al norte con los Estados Unidos de Amé...,3946.5,2.2,23.3
Alamos,26.383333,27.783333,108.416667,109.25,50.0,2000.0,Colinda al norte con el municipio de Quiriego ...,6422.8,3.6,3.9
Altar,30.516667,31.666667,111.6,112.283333,200.0,1700.0,Colinda al norte con el municipio de Caborca y...,4457.7,2.5,2.1
Arivechi,28.733333,28.983333,108.833333,109.266667,400.0,2200.0,"Colinda al norte, este, sur y oeste con el mun...",726.3,0.4,1.6
...,...,...,...,...,...,...,...,...,...,...
Villa Pesqueira,28.783333,29.5,109.783333,110.133333,50.0,1700.0,Colinda al norte con los municipios de Baviáco...,1123.2,0.6,0.9
Yécora,28.166667,28.65,108.466667,110.133333,300.0,2300.0,Colinda al norte con los municipios de Bacanor...,2667.7,1.5,1.8
Gral. Plutarco Elías Calles,31.316667,32.083333,112.3,113.516667,100.0,1400.0,Colinda al norte con el municipio de Puerto Pe...,3656.7,2,3.7
Benito Juárez,26.966667,27.233333,109.716667,109.983333,0.0,100.0,Colinda al norte con los municipios de Cajeme ...,369.5,0.2,58.7
